# Assignment 2: Exercises 4.1-4.3, 5.1-5.2
Aaron Ou | 2018-02-19 updated on 2018-03-18

### Exercise 4.1:

<!--Open PanelPROGRESA_97_99year.csv. This dataset is a repeated cross-
section of different waves of the ENCEL survey for March and October 1998 and November 1999. It
also includes the baseline data collected in 1997. We have used this or part of this dataset previous
modules as well.

Let’s assume that we expect PROGRESA (D_HH) to change the income of the household
(IncomeLabHH) in 1999. Specify a regression model in R as discussed above and restrict the data to
be used from year 1999 (variable year). What is the impact of the intervention on income levels? Is
it statistically significant? How do you interpret the coefficient?
-->

In [1]:
# Clear environment 
rm(list = ls())

# Load packages
suppressMessages(library(dplyr))
suppressMessages(library(miceadds))
suppressMessages(library(multiwayvcov))
suppressMessages(library(gmodels))
suppressMessages(library(AER))

In [2]:
d1 <- read.csv("../Data/PanelPROGRESA_97_99year.csv") %>% filter(year == 1999)
names(d1)

[1] "year"         "villid"       "geopolid"     "hogid"        "pov_HH"      
 [6] "D"            "D_HH"         "IncomeLab_HH" "famsize"      "eduhead"     
[11] "agehead"      "sexhead"

In [3]:
model0 <- lm(data = d1, IncomeLab_HH ~ D_HH, na.action = 'na.exclude')
summary(model0)


Call:
lm(formula = IncomeLab_HH ~ D_HH, data = d1, na.action = "na.exclude")

Residuals:
    Min      1Q  Median      3Q     Max 
  -1977   -1094    -699     -99 1801006 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1898.63     144.12  13.174   <2e-16 ***
D_HH           94.89     238.57   0.398    0.691    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 15570 on 18368 degrees of freedom
  (4893 observations deleted due to missingness)
Multiple R-squared:  8.612e-06,	Adjusted R-squared:  -4.583e-05 
F-statistic: 0.1582 on 1 and 18368 DF,  p-value: 0.6908


The p-value of the coefficient of `D_HH` is 0.69, much bigger than 0.05 (95% confidence level). Thus, we fail to reject H0. The correlation between income and the PROGRESA(`D_HH`) is not statistically significant.

### Exercise 4.2:

<!--
Conduct a two-sided t-test to compare if household income is different by the treatment
assignment. Are the results same as those from the regression analysis?
-->

In [4]:
# Calculate t-statistics
t1 <- t.test(IncomeLab_HH ~ D_HH, data = d1, var.equal = T)
t1
t_value <- round(t1$statistic, 2)


	Two Sample t-test

data:  IncomeLab_HH by D_HH
t = -0.39773, df = 18368, p-value = 0.6908
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -562.5125  372.7393
sample estimates:
mean in group 0 mean in group 1 
       1898.628        1993.514 


In [5]:
t1$parameter

df 
18368

In [6]:
# Calculate critical t-value at 90% confidence level; 5% significance level for a two sided t-test
crit <- round(qt(0.05, t1$parameter), 2)
crit

[1] -1.64

In [7]:
# Compare t-statistic with critical t-value
print(paste0("The calculated t-value is ", t_value, 
             ", greater than the critical t-value ",
             crit) 
      )
paste0(round(t1$statistic, 2), " > ", crit)

[1] "The calculated t-value is -0.4, greater than the critical t-value -1.64"


[1] "-0.4 > -1.64"

Therefore, we fail to reject H0, say that the difference is not significnat enough. The result is the same with those from the regression analysis.

### Exercise 4.3:

<!--
As a demonstration, let’s assume that poverty status of a household (pov_HH) is a
confounder and we should control for it in the regression analysis. Specify a regression model in R that
tests the effect of PROGRESA on income, controlling for poverty status of the household and
remembering to restrict the data to be used from year 1999 (variable year).
Now, specify a separate regression model, this time clustering the standard errors at the village level.
We can most easily accomplish this using the lm.cluster function in R:
PROGRESA_99 <- PanelPROGRESA_97_99year[which(PanelPROGRESA_97_99year$year
== 1999), ]
model2 <- lm.cluster(data = PROGRESA_99, formula=IncomeLab_HH ~ D_HH+pov_HH,
cluster=PROGRESA_99$villid)
summary(model2)
You will need to install and load the packages miceadds and multiwayvcov to use this command.
What happens to the effect size (magnitude of the coefficient) and the standard errors when
comparing output from the two models? Based on the output from these regressions, did PROGRESA
significantly impact income levels?
-->

In [8]:
model1 <- lm(data = d1, IncomeLab_HH ~ D_HH + pov_HH)
summary(model1)


Call:
lm(formula = IncomeLab_HH ~ D_HH + pov_HH, data = d1)

Residuals:
    Min      1Q  Median      3Q     Max 
  -2188   -1094    -714     -31 1801006 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   2204.4      252.4   8.735   <2e-16 ***
D_HH           229.0      266.0   0.861    0.389    
pov_HHpoor    -440.0      312.2  -1.409    0.159    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 15750 on 17939 degrees of freedom
  (5321 observations deleted due to missingness)
Multiple R-squared:  0.0001162,	Adjusted R-squared:  4.757e-06 
F-statistic: 1.043 on 2 and 17939 DF,  p-value: 0.3525


In [9]:
# cluster by village
model2 <- lm.cluster(data = d1, IncomeLab_HH ~ D_HH + pov_HH, cluster = d1$villid)
summary(model2)

R^2 = 0.00012 

             Estimate Std. Error    t value      Pr(>|t|)
(Intercept) 2204.4456   75.01622 29.3862515 8.230176e-190
D_HH         229.0350  324.93091  0.7048728  4.808894e-01
pov_HHpoor  -439.9662  138.20071 -3.1835308  1.454906e-03


Comparing `model1` and `model2`, while the magnitude of the coefficient remains almost the same (229), the standard error increases from 266 in `model1` to 324 in the clustered `model2`.

However, both `model1` and `model2` fail to reject H0 at 90% confidence level:


Model | P-value of D_HH | Comment
--- | --- | ---
`model1` | 0.389　| > 0.1
`model2` | 0.48 | > 0.1

Based on the output from the two regressions, PROGRESA did not impact income levels significantly.

### Exercise 5.1:

<!--
Conduct a two-sided t-test to compare if household income is different by the treatment
assignment. Are the results same as those from the regression analysis?Open the PanelPROGRESA_97_99year.csv; this is the same dataset as that
used previously. We assume that the villages (villid) were randomly assigned PROGRESA (D).
However, the families in the study villages had a choice whether to participate in the program; we
indicate participants with D_HH. In reality there was an eligibility criterion to participate in
PROGRESA in the selected villages, but for this exercise we will assume that all households in
treatment villages were eligible. Now, cross-tabulate the frequency of households assigned to the
treatment groups and their actual participation. What is the level of compliance?
Now estimate the effect of D and D_HH on household income (IncomeLab_HH) in year 1998. Which
effect is larger in magnitude (ignore their statistical significance)? Do you think that either effect is
measured causally?
-->

In [10]:
d2 <- read.csv("../Data/PanelPROGRESA_97_99year.csv")
head(d2, 2)
unique(d2$D_HH)

year,villid,geopolid,hogid,pov_HH,D,D_HH,IncomeLab_HH,famsize,eduhead,agehead,sexhead
1998,13030105,13,1,poor,Treated,NA,NA,2,NA,70,Male
1998,13030024,13,2,Non poor,Control,0,9000,3,NA,27,Male


[1] NA  0  1

In [11]:
d2_trt <- d2 %>% filter(D == 'Treated') %>% select(D, D_HH) %>% na.omit

##### Level of compliance = Actual participation / Assigned to Treat

In [12]:
paste0("The level of compliance in treatment group is ",
       nrow(filter(d2_trt, D_HH == 1)), " / ", nrow(d2_trt), " = ",
    round(nrow(filter(d2_trt, D_HH == 1))*100 / nrow(d2_trt)), "%")

[1] "The level of compliance in treatment group is 15011 / 41840 = 36%"

Or use `CrossTable` function from `gmodels` package:

In [13]:
CrossTable(d2_trt$D, d2_trt$D_HH, prop.t = TRUE, prop.chisq = FALSE)


 
   Cell Contents
|-------------------------|
|                       N |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  41840 

 
             | d2_trt$D_HH 
    d2_trt$D |         0 |         1 | Row Total | 
-------------|-----------|-----------|-----------|
     Treated |     26829 |     15011 |     41840 | 
             |     0.641 |     0.359 |           | 
-------------|-----------|-----------|-----------|
Column Total |     26829 |     15011 |     41840 | 
-------------|-----------|-----------|-----------|

 


The result is the same, 0.359 (36%)

##### Effect of `D` and `D_HH` on `IncomeLab_HH` in year 1998

In [14]:
model3 <- lm(data = filter(d2, year == 1998), IncomeLab_HH ~ D)
summary(model3)


Call:
lm(formula = IncomeLab_HH ~ D, data = filter(d2, year == 1998))

Residuals:
   Min     1Q Median     3Q    Max 
 -1725   -890   -627    -27 748266 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   1733.6      119.3  14.537   <2e-16 ***
DTreated      -206.2      152.5  -1.352    0.176    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 9942 on 17877 degrees of freedom
  (6192 observations deleted due to missingness)
Multiple R-squared:  0.0001023,	Adjusted R-squared:  4.633e-05 
F-statistic: 1.828 on 1 and 17877 DF,  p-value: 0.1763


In [15]:
model4 <- lm(data = filter(d2, year == 1998), IncomeLab_HH ~ D_HH)
summary(model4)


Call:
lm(formula = IncomeLab_HH ~ D_HH, data = filter(d2, year == 1998))

Residuals:
   Min     1Q Median     3Q    Max 
 -1780   -889   -517      8 748211 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   1788.7       92.5  19.337  < 2e-16 ***
D_HH          -521.2      163.3  -3.193  0.00141 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 10060 on 17431 degrees of freedom
  (6638 observations deleted due to missingness)
Multiple R-squared:  0.0005844,	Adjusted R-squared:  0.0005271 
F-statistic: 10.19 on 1 and 17431 DF,  p-value: 0.001412


**Which effect is larger in magnitude?**

The coefficient of `D_HH` is larger than that of `D` in magnitude.
 
**Are the two effects measured causally?**

At 90% confidence level, the p-value of `D` is 0.48 (> 0.1), not statistically significant; the p-value of `D_HH` is 0.002 (< 0.1), statistically significant.    

1. `D`(ITT) is measured causally because the selection of treatment or control is supposed to be random;
2. `D_HH`(TOT) is note measured causally because those who actually take up PROGRESA may be the poorer families, with lower income at the baseline. There is selection bias.

### Exercise 5.2:

<!--
In R, specify a 2SLS model to estimate the ATET or TOT effects of PROGRESA
participation on household income levels in 1999. The IV for D_HH should be D. Please read the
help for command ivreg. Now use D and pov_HH as IVs for D_HH and estimate the TOT/ATET. Are the
results same or different? Why?
-->

In [16]:
# simple linear regression
summary(lm(IncomeLab_HH ~ D_HH, data = d1))


Call:
lm(formula = IncomeLab_HH ~ D_HH, data = d1)

Residuals:
    Min      1Q  Median      3Q     Max 
  -1977   -1094    -699     -99 1801006 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1898.63     144.12  13.174   <2e-16 ***
D_HH           94.89     238.57   0.398    0.691    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 15570 on 18368 degrees of freedom
  (4893 observations deleted due to missingness)
Multiple R-squared:  8.612e-06,	Adjusted R-squared:  -4.583e-05 
F-statistic: 0.1582 on 1 and 18368 DF,  p-value: 0.6908


In [17]:
# `ivreg` function from `AER` package
# use D as IV for D_HH
iv_model1 <- ivreg(IncomeLab_HH ~ D_HH | D, data = d1)
summary(iv_model1)


Call:
ivreg(formula = IncomeLab_HH ~ D_HH | D, data = d1)

Residuals:
       Min         1Q     Median         3Q        Max 
  -2101.33   -1077.09    -630.50     -27.09 1800882.00 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   1827.1      176.1  10.378   <2e-16 ***
D_HH           290.9      365.6   0.796    0.426    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 15570 on 18368 degrees of freedom
Multiple R-Squared: -2.814e-05,	Adjusted R-squared: -8.258e-05 
Wald test: 0.633 on 1 and 18368 DF,  p-value: 0.4263 


In [18]:
# use D and pov_HH as IVs for D
iv_model2 <- ivreg(IncomeLab_HH ~ D_HH | D + pov_HH, data = d1)
summary(iv_model2)


Call:
ivreg(formula = IncomeLab_HH ~ D_HH | D + pov_HH, data = d1)

Residuals:
      Min        1Q    Median        3Q       Max 
  -1949.9   -1052.0    -734.8    -134.8 1801065.2 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1951.98     166.72  11.708   <2e-16 ***
D_HH          -17.23     316.33  -0.054    0.957    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 15750 on 17940 degrees of freedom
Multiple R-Squared: -2.77e-06,	Adjusted R-squared: -5.851e-05 
Wald test: 0.002966 on 1 and 17940 DF,  p-value: 0.9566 


Comparing the two models, the results are different. Ignoring the statistical significance, in `iv_model1`, the coefficient of `D_HH` is 290 while that in `iv_model2` is -17.23.

`pov_HH` is a bad instrumental variable (IV) in `iv_model2`. It is hard to remove the confounding factors.